In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

from scipy import stats

In [2]:
xlim     = 20. # ms
fs       = 25. # kHz

file_dir = '../Step_1/'

In [3]:
DIVs     = [17,19,21,23,27]
circuits = list(np.arange(15))
MEAs     = [2,3,4,6,7,8,9,10]

In [4]:
def get_pattern(DIVs,circuit_stim,circuit_record):
    
    num_stimuli   = []
    spikes        = []
    
    for i in range(4):
        spikes.append([])
        num_stimuli.append([])
        for j in range(4):
            spikes[i].append([])
            for k in range(len(DIVs)):
                spikes[i][j].append([])
                if j == 0:
                    num_stimuli[i].append([])
    
    for k,DIV in enumerate(DIVs):
        f = h5py.File(file_dir + 'MEA_' + str(MEA) + '_DIV_' + str(DIV) + '.h5','r')
        circuit_group = f['Circuit_'+str(circuit_record)]
        for i in range(4):
            pattern_group = circuit_group['Pattern_'+str(i+10+4*circuit_stim)]

            stim_ids      = np.array(pattern_group['Stim_IDs'])
            num_stimuli[i][k] = stim_ids.shape[0]

            s             = np.array(pattern_group['Spikes'])[:,0,:]

            for j in range(4):
                temp_spikes     = (s[s[:,1]==j,0:3:2])
                if circuit_stim == circuit_record and i == j:
                    # This means, the first spike needs to be removed (stim artifact)
                    mask = np.argwhere(np.concatenate([np.ones(1),temp_spikes[1:,0]-temp_spikes[:-1,0]],0)==0)[:,0]
                    temp_spikes = temp_spikes[mask,:]
                spikes[i][j][k] = temp_spikes
        f.close()
        
    for i in range(4):
        num_stimuli[i] = np.sum(np.array(num_stimuli[i]))
        for j in range(4):
            spikes[i][j] = np.concatenate(spikes[i][j],axis=0)
        
    return spikes,num_stimuli

In [5]:
def uniform_cdf(x):
    # This is the true uniform distribution that we would expect, if spiking is independent from stimulus
    return np.maximum(np.minimum(x/200.,1),0)

In [6]:
def get_p_value(DIVs,circuit_stim,circuit_record):    
    spikes,num_stimuli = get_pattern(DIVs,circuit_stim,circuit_record)
    
    p_values = np.zeros((4,4)) + 1
    for i in range(4):
        for j in range(4):
            s             = np.array(spikes[i][j])/fs
            s             = s[s[:,1]<200,1]
            s             = s
            if s.shape[0] == 0:
                continue
            _,p_value     = stats.kstest(s,uniform_cdf)
            p_values[i,j] = p_value
    return p_values

In [7]:
f_out    = h5py.File('../Step_4/step_4_activity_independence.h5','w')
for MEA in MEAs:
    print(MEA)
    p_values = np.zeros((4*len(circuits),4*len(circuits)))
    for i in range(len(circuits)):
        for j in range(len(circuits)):
            p_values[i*4:(i+1)*4,j*4:(j+1)*4] = get_p_value(DIVs,circuits[i],circuits[j])
    f_out.create_dataset(name = 'MEA_'+str(MEA),data=p_values)
f_out.close()

2
3
4
6
7
8
9
10
